比赛名称:NCAA2024-中文糖尿病问题分类评测

比赛链接:https://tianchi.aliyun.com/competition/entrance/532176/information

这里使用谷歌新出的Gemma模型作为baseline.

# github 常用命令

(https://zhuanlan.zhihu.com/p/637271164?utm_id=0)


git init

git config user.email "helloworld@hh.com"

git config user.name "yunsuxiaozi"

git add .

git commit -m "202401LWP2WQ"

git branch -M main

git remote add 202401LWP2WQ https://github.com/yunsuxiaozi/AI-and-competition.git

git push -u 202401LWP2WQ main

###   安装必要的库

In [1]:
import os#与操作系统进行交互的库
os.environ["TOKENIZERS_PARALLELISM"] = "false"#transformer的分词器串行使用(不是并行)
#PEFT（Parameter-Efficient Fine-Tuning）库是一个用于高效微调预训练语言模型（PLMs）的工具库
!pip install peft==0.8.2
#处理二进制数据的库，它提供了一种简单的方式来处理字节和位操作
!pip install -U -q bitsandbytes
#简化分布式和多GPU训练
!pip install -U -q accelerate
#用于加载、处理和操作数据集的库
!pip install datasets==2.16.1
#包含了大量预训练的自然语言处理模型
!pip install transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successf

### 导入必要的库

In [2]:
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
from tqdm import tqdm#进度条库

import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。
#在transformer中设置随机种子,序列分类任务的模型
from transformers import set_seed,AutoModelForSequenceClassification
#自动分词器可以根据指定的模型确定分词方法,提供填充截断功能,训练模型的类,配置训练参数的类
from transformers import AutoTokenizer,DataCollatorWithPadding, Trainer, TrainingArguments
#允许你快速创建一个管道,用户指定任务,库会自动加载相应的预训练模型，并提供一个接口来执行这个任务
from transformers import pipeline

#prepare_model_for_int8_training:将模型的权重从float32变成int8
#LoraConfig:用于设置LoRA（Low-Rank Adaptation）微调的参数
#TaskType:枚举类型,如文本分类（SEQ_CLS）、序列到序列的生成任务（SEQ_2_SEQ_LM）、因果语言建模（CAUSAL_LM）
#get_peft_model:获取经过PEFT微调的模型
from peft import prepare_model_for_int8_training,LoraConfig, TaskType, get_peft_model

2024-03-09 07:29:00.437163: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 07:29:00.437265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 07:29:00.563111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### 设置基础的参数.

In [3]:
SEED = 2024#设置随机种子
NUM_CLASSES = 6#分类任务的类别数
EPOCHS=5#模型微调的次数
R=16#lora中低秩矩阵的秩
LORA_ALPHA=16#缩放因子，用于控制 LoRA 适配器中新添加参数的影响
LORA_DROPOUT=0.1#lora适配器中dropout的比例
BATCH_SIZE = 2#每个句子长度不一样,可能有些句子特别长,导致batch_size太大会超内存.
MODEL_ID = '/kaggle/input/gemma/transformers/2b/2'#预训练模型和分词器的路径
file_path="/kaggle/input/2024ali/2024ali/"
set_seed(SEED)#transformer模型中设置随机种子

### 导入训练数据

In [4]:
train_data=[]
train_label=[]
with open(file_path+"train.txt",'r') as f:
    data=f.readline()
    while data:
        if len(data.split())!=2:#正常来说有数据和标签
            pass
        else:
            data,label=data.split()
            train_data.append(data)
            train_label.append(int(label))
        data=f.readline()
with open(file_path+"dev.txt",'r') as f:
    data=f.readline()
    while data:
        if len(data.split())!=2:#正常来说有数据和标签
            pass
        else:
            data,label=data.split()
            train_data.append(data)
            train_label.append(int(label))
        data=f.readline()
#len(data.split())!=2的数据        
dataset=[['绝经期女性是否容易得糖尿病?', '2'],
['频繁urticaria是否是糖尿病的症状?', '4'],
['为什么糖尿病人不适合缺医院治疗?', '5']]
for idx in range(len(dataset)):
    data,label=dataset[idx]
    train_data.append(data)
    train_label.append(int(label))

print(f"len(train_label):{len(train_label)}")

train_df=pd.DataFrame({"text":train_data,"label":train_label})
train_df.to_csv("train_df.csv",index=None)

len(train_label):11000


### train_test_split

In [5]:
#用于加载数据集
from datasets import load_dataset
#加载的是csv文件,文件路径为train.csv
dataset = load_dataset('csv', data_files="train_df.csv")
#将数据划分成训练数据和测试数据,检验真实模型的泛化性
train_size=10000#选择10000条数据作为训练数据
dataset['test'] = dataset['train'].select(range(train_size,len(dataset['train'])))
dataset['train']=dataset['train'].select(range(train_size))

Generating train split: 0 examples [00:00, ? examples/s]

### 导入分词器.

In [6]:
#导入gemma模型对应的预训练分词器
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
#对训练数据和测试数据进行分词的操作.
tokenized_dataset = {}
for split in ['train','test']:
    tokenized_dataset[split] = dataset[split].map(
        #truncation=True 如果文本长度超过模型最大长度,就进行截断操作.批处理执行分词操作.
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )  

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### 序列分类模型.

In [7]:
#加载预训练的序列分类模型,模型选择是gemma,分类的类别数为2.
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=NUM_CLASSES,
)
#将模型的权重从float32变成int8.
model = prepare_model_for_int8_training(model)
"""
Low-Rank Adaptation of Large Language Models 低秩自适应大语言模型
我的理解 原始参数是W0,现在引入低秩矩阵BA,W0参数冻结,(W0+BA)x=y,训练BA 
比如 W0:(100,100) B:(100,10) A:(10,100),这样参数变少了.
"""
lora_config = LoraConfig(
    r=R,#低秩矩阵的秩
    lora_alpha=LORA_ALPHA,#缩放因子，用于控制 LoRA 适配器中新添加参数的影响
    lora_dropout=LORA_DROPOUT,#lora适配器中dropout的比例
    task_type=TaskType.SEQ_CLS,#任务类型是序列分类任务
    target_modules='all-linear'#'all-linear' 表示适配器将应用于模型中所有的线性层.
)

#根据原始模型的结构和参数添加参数来做下游任务,比如知道W0得到BA.
lora_model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/gemma/transformers/2b/2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 模型的训练.

In [8]:
#计算评估指标的函数,准确率.
def compute_metrics(eval_pred):
    predictions, labels = eval_pred#传出的是预测的概率和标签
    predictions = np.argmax(predictions, axis=1)#预测结果转成标签
    return {"accuracy": (predictions == labels).mean()}#计算预测的准确率.

trainer = Trainer(
    model=lora_model,#模型是低秩自适应大语言模型.
    args=TrainingArguments(
        output_dir="./data/",#保存模型和训练日志的位置
        learning_rate=2e-5,#学习率设置
        per_device_train_batch_size=BATCH_SIZE,#每次训练的数据量
        per_device_eval_batch_size=BATCH_SIZE,#每次推理的数据量
        evaluation_strategy="epoch",#每个训练周期后进行评估
        save_strategy="epoch",#每个epoch结束后保存一次
        num_train_epochs=EPOCHS,#训练多少次
        weight_decay=0.01,#权重衰退,控制模型更新的正则化项
        load_best_model_at_end=True,#最终加载最好的模型
        logging_steps=10,#训练日志输出的步数间隔
        report_to="none",#不输出训练进度的报告
    ),
    #训练数据和评估数据
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,#分词器
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),#数据截断填充器
    compute_metrics=compute_metrics,#评估指标是准确率
)
print("Training the Model")
trainer.train()#模型训练
print("Saving the model!")
#创建"fine-tuned_model"的文件夹,并在其中保存模型的配置文件和权重文件。
lora_model.save_pretrained('fine-tuned-model')

Training the Model


Epoch,Training Loss,Validation Loss,Accuracy
1,0.544800,1.730109,0.727000
2,0.583000,1.856172,0.727000
3,0.185100,2.027011,0.725000
4,0.006700,2.092614,0.720000
5,0.723300,2.127344,0.716000


Saving the model!


### 模型的预测.

In [9]:
#我们想创建文本分类的管道,传入创建的模型和分词器
clf = pipeline("text-classification", lora_model, tokenizer=MODEL_ID)
# #导入测试数据和提交文件
test_data=[]
with open("/kaggle/input/2024ali/2024ali/test.txt",'r') as f:
    data=f.readline()
    while data:
        test_data.append(data)
        data=f.readline()
test_df=pd.DataFrame({"text":test_data})

print("Making prediction on test dataset...")

predictions = []
for text in tqdm(test_df['text'].values):
    prediction = clf(text)
    prediction = int(prediction[0]['label'].split('_')[1])
    predictions.append(prediction)
for  idx in range(len(predictions)):
    print(predictions[idx])

The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification'

Making prediction on test dataset...


100%|██████████| 2000/2000 [1:08:08<00:00,  2.04s/it]

0
0
4
3
3
5
2
2
1
1
2
5
3
1
0
1
2
1
4
3
2
2
1
1
1
3
2
3
2
3
4
1
1
1
1
2
4
3
2
1
1
1
3
1
2
0
2
2
1
1
1
3
2
1
2
1
2
2
4
3
2
2
4
2
2
1
3
2
3
4
2
5
2
2
2
2
1
2
2
1
2
2
2
1
4
3
3
1
2
1
0
2
3
3
3
3
3
0
3
3
1
2
1
3
3
2
3
1
3
3
2
0
3
5
3
4
3
2
1
1
1
3
4
3
2
2
1
1
4
2
0
1
1
1
4
1
3
1
3
3
2
1
1
2
2
1
2
1
4
3
5
5
2
0
3
1
3
1
0
1
0
3
2
2
1
5
1
5
1
3
2
5
4
4
3
3
4
2
4
1
2
4
1
2
1
0
1
1
2
2
1
3
2
1
3
1
3
0
2
2
2
2
1
1
2
3
3
0
0
3
1
1
1
3
3
1
2
3
3
0
4
3
3
4
2
1
2
0
1
4
2
2
1
2
1
1
3
2
0
2
3
1
1
2
3
1
3
3
1
2
1
2
2
2
3
3
1
3
1
2
1
2
1
2
3
3
2
1
2
3
2
2
3
1
3
1
2
3
2
3
2
0
2
0
3
2
2
3
3
3
1
2
2
4
1
2
2
2
3
3
2
4
3
0
3
2
0
3
0
2
1
3
0
2
3
0
2
3
3
3
0
2
1
2
3
2
4
2
1
1
4
2
1
2
3
0
4
1
4
2
2
2
3
2
3
3
1
3
2
3
3
3
3
3
3
3
2
3
3
1
2
2
1
2
3
3
2
1
2
3
2
2
1
3
1
2
3
1
1
2
2
3
1
2
5
2
3
4
2
3
1
1
3
1
1
2
2
1
2
1
2
1
3
3
5
1
3
3
1
2
5
0
2
2
2
3
2
3
0
2
3
2
2
3
1
1
2
2
2
1
3
2
0
1
2
1
2
3
2
1
1
3
3
1
0
2
2
2
1
1
3
3
1
3
1
2
3
4
2
2
2
0
2
2
3
0
3
3
1
3
1
1
4
2
1
1
2
1
0
1
2
2
1
2
5
5
2
1
2
4
2
4
3
2
1
3
4
3
0
2


### 预测结果的保存.

In [10]:
#这个是能够成功提交的代码
df_test = pd.read_csv(file_path+'test.txt', sep='\t', header=None)
df_test.columns = ['text']

df_test['labels'] = predictions
df_test.to_csv("google_gemma_baseline.txt", sep='\t', index=False, header=False)